Start by importing necessary modules.

In [176]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Then we'll import the unclean data.

In [177]:
df1 = pd.read_csv('../../data/imdb.title.basics.csv.gz')
df2 = pd.read_csv('../../data/bom.movie_gross.csv.gz')
df3 = pd.read_csv('../../data/tn.movie_budgets.csv.gz')
df4 = pd.read_csv('../../data/imdb.title.ratings.csv.gz')

We're going to concatenate df2 and df3, then merge the concatenated table with df1 and df4. Let's start by cleaning df1 and df4.

We need to clean df1 by dropping foreign title column, runtime, renaming some columns, and dropping any entries which don't have genre data.

In [178]:
df1 = df1.drop(['original_title', 'runtime_minutes'], axis=1)
df1 = df1.rename(columns={'primary_title':'title', 'start_year': 'year'})
indices_without_genre_data = df1[df1['genres'].isna()].index
df1 = df1.drop(indices_without_genre_data)

We want to make new bool-datatype columns for each genre, and split the data currently in the genres column. So we defined a function for that, by using a nested for-loop, and the convenience that the python `set` object automatically does not add duplicates.

In [179]:
def sep_genre(genres):
    unique_genre = set()
    for genre in genres:
        if type(genre) == str:
            genre_list = genre.split(',')
            for g in genre_list:
                unique_genre.add(g)
    return unique_genre

genres_col_names = list(sep_genre(df1['genres']))
for genre in genres_col_names:
    df1[genre] = df1.genres.str.contains(pat = genre).astype(bool)

Now, we just delete any movies that haven't come out yet and any duplicates.

In [180]:
indices_before_2020 = df1[df1['year'] >= 2020].index
df1 = df1.drop(indices_before_2020)

df1 = df1.drop_duplicates(subset='title')

Finally, drop unnecessary 'tconst' and 'year' columns.

In [181]:
df1 = df1.drop(['year'], axis=1)

genre_df = df1
genre_df.head()

,tconst,title,genres,Animation,Game-Show,War,Musical,Family,Sci-Fi,Drama,...,Documentary,Crime,News,Fantasy,Reality-TV,History,Horror,Romance,Comedy,Sport
0,tt0063540,Sunghursh,"Action,Crime,Drama",False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
1,tt0066787,One Day Before the Rainy Season,"Biography,Drama",False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,tt0069049,The Other Side of the Wind,Drama,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,tt0069204,Sabse Bada Sukh,"Comedy,Drama",False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
4,tt0100275,The Wandering Soap Opera,"Comedy,Drama,Fantasy",False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,True,False


Next,df4. It's already clean enough for our purposes, so we move onto preparing df2 and df3 for concatenation.

In [182]:
ratings_df = df4

For df2, we needed to get it down to a dataframe of only 4 columns, 'title' (object), 'dom_gross' (numeric), 'for_gross' (numeric), and 'year' (numeric). We managed to do that, but apparently the original data file was overwritten. See the commented code below for an explanation of how.

In [183]:
#df2 = df2.rename(columns={'domestic_gross':'dom_gross', 'foreign_gross': 'for_gross' })
# df2['foreign_gross'] = pd.to_numeric(df2['foreign_gross'], errors='coerce')
df2 = df2.drop(columns='studio')
# df2 = df2.drop(df2[(df2['dom_gross'] == 'NaN') & (df2['for_gross'] == 'NaN')].index)
# cleaned_df2 = df2
# cleaned_df2 = cleaned_df2.drop_duplicates(subset='title', keep=False)

bom = df2
bom.head()

,title,domestic_gross,foreign_gross,year
0,Toy Story 3,415000000.0,652000000,2010
1,Alice in Wonderland (2010),334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,296000000.0,664300000,2010
3,Inception,292600000.0,535700000,2010
4,Shrek Forever After,238700000.0,513900000,2010


For df3, we just have to get it to the same format as listed above.

In [184]:
df3_cleaned = df3.drop(['id', 'production_budget'], axis=1)

In [185]:
# pull out only the year from release_date and coerce to int type
df3_cleaned['release_date'] = [date[-4:] for date in df3_cleaned['release_date']]
df3_cleaned['release_date'] = df3_cleaned['release_date'].astype(int)

In [186]:
# strip any '$' and ',' from domestic/worldwide gross, then coerce to int
df3_cleaned['domestic_gross'] = [amount.replace('$', '').replace(',','') for amount in df3_cleaned['domestic_gross']]
df3_cleaned['worldwide_gross'] = [amount.replace('$', '').replace(',','') for amount in df3_cleaned['worldwide_gross']]
df3_cleaned['domestic_gross'] = df3_cleaned['domestic_gross'].astype(int)
df3_cleaned['worldwide_gross'] = df3_cleaned['worldwide_gross'].astype(int)

In [187]:
# make worldwide gross only track foreign revenue by subtracting domestic values from it
df3_cleaned['worldwide_gross'] = [df3_cleaned['worldwide_gross'][i] - df3_cleaned['domestic_gross'][i] for i in range(len(df3_cleaned['worldwide_gross']))]

In [188]:
# rename cols
df3_cleaned = df3_cleaned.rename(columns = {'movie': 'title', 'release_date': 'year', 'worldwide_gross': 'for_gross', 'domestic_gross': 'dom_gross'})

# rearrange cols
df3_cleaned = df3_cleaned[['title', 'year', 'dom_gross', 'for_gross']]

tn = df3_cleaned
tn.head()

,title,year,dom_gross,for_gross
0,Avatar,2009,760507625,2015837654
1,Pirates of the Caribbean: On Stranger Tides,2011,241063875,804600000
2,Dark Phoenix,2019,42762350,107000000
3,Avengers: Age of Ultron,2015,459005868,944008095
4,Star Wars Ep. VIII: The Last Jedi,2017,620181382,696540365


Now, concatenate bom and tn and clean a little bit more.

In [189]:
concat_df = pd.concat([bom, tn], ignore_index=True, sort=True)

In [190]:
concat_df = concat_df.drop_duplicates(subset='title')
concat_df = concat_df.drop(columns=['dom_gross', 'for_gross'])
#concat_df = concat_df[['title', 'year', 'dom_gross', 'for_gross']]

We made a new feature here of worldwide gross, and also deleted any missing values of domestic_gross.

In [191]:
world_gross = []

for domestic, foreign in zip(concat_df.dom_gross, concat_df.for_gross):
    if np.isnan(foreign):
        world_gross.append(np.nan)
    else:
        world_gross.append(domestic + foreign)
        
concat_df['world_gross'] = pd.Series(world_gross, index=concat_df.index)

AttributeError: 'DataFrame' object has no attribute 'dom_gross'

In [ ]:
concat_df = concat_df.drop(concat_df[(concat_df['dom_gross'] == 0.0) | (concat_df['for_gross'] == 0.0)].index)

In [192]:
gross_df = pd.read_csv('../../data/bom_imdb_revenue_concatenated.csv')

In [ ]:
#genre_df = genre_df.drop(['Unnamed: 0'], axis=1)
#gross_df = gross_df.drop(['Unnamed: 0'], axis=1)
#ratings_df = ratings_df.drop(['Unnamed: 0'], axis=1)

In [193]:
ratings_df.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [194]:
full_df = ratings_df.merge(genre_df, on='tconst')

Now we drop some genre columns with too little representation.

In [195]:
full_df = full_df.drop(['Talk-Show', 'Adult', 'Game-Show', 'Short', 'News', 'Reality-TV', 'tconst', 'Musical', 'Western'], axis=1)

In [196]:
full_gross = full_gross.drop(full_gross[(full_gross['dom_gross'].isnull())].index)

NameError: name 'full_gross' is not defined